__Description & purpose__:    

__Author(s)__: Alastair Graham, Dusan Figala

__Date created__: 2024-09-10

__Date last modified__: 2024-12-05

__Licence__: This notebook is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  The code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# ADD Oxidian link

# What this does
This notebook demostrates usage of the EODH ADES API using pyeodh.

EODH provides Application Deployment & Execution Service - ADES, to which you can deploy workflows and execute parametrised processing jobs. Pyeodh provides an interface to simplify interaction with ADES from python scripts.

First we need to instantiate pyeodh client and get the ADES entrypoint.

Note: This API requires authentication credentials to be provided by the user (in this case read from environment variables). This is a subject to change as the hub is implementing proper IAM solution.

In [ ]:
from requests import HTTPError
import os
from pprint import pp
from dotenv import load_dotenv

import pyeodh

from pathlib import Path

dotenv_path = Path('reqts/.env')
load_dotenv(dotenv_path=dotenv_path)

#load_dotenv()
username = os.getenv("ADES_USER")
token = os.getenv("ADES_TOKEN")


client = pyeodh.Client(username=username, token=token)
ades = client.get_ades()



In [22]:
!unset ADES_TOKEN

In [2]:
for name, value in os.environ.items():
    print("{0}: {1}".format(name, value))

CHROME_DESKTOP: code.desktop
CONDA_DEFAULT_ENV: eodh
CONDA_EXE: /home/al/miniforge3/bin/conda
CONDA_PREFIX: /home/al/miniforge3/envs/eodh
CONDA_PROMPT_MODIFIER: (eodh) 
CONDA_PYTHON_EXE: /home/al/miniforge3/bin/python
CONDA_SHLVL: 2
DBUS_SESSION_BUS_ADDRESS: unix:path=/run/user/1000/bus
DESKTOP_SESSION: pop
DISPLAY: :1
GDK_BACKEND: x11
GDMSESSION: pop
GIO_LAUNCHED_DESKTOP_FILE: /usr/share/applications/code.desktop
GIO_LAUNCHED_DESKTOP_FILE_PID: 5484
GJS_DEBUG_OUTPUT: stderr
GJS_DEBUG_TOPICS: JS ERROR;JS LOG
GNOME_DESKTOP_SESSION_ID: this-is-deprecated
GNOME_SHELL_SESSION_MODE: pop
GPG_AGENT_INFO: /run/user/1000/gnupg/S.gpg-agent:0:1
GSETTINGS_SCHEMA_DIR: /home/al/miniforge3/envs/eodh/share/glib-2.0/schemas
GTK_IM_MODULE: ibus
GTK_MODULES: gail:atk-bridge
HOME: /home/al
INVOCATION_ID: 051e25278c044f50b26e0cc2a4a7b8ea
JOURNAL_STREAM: 9:18828
LANG: en_GB.UTF-8
LESSCLOSE: /usr/bin/lesspipe %s %s
LESSOPEN: | /usr/bin/lesspipe %s
LOGNAME: al
MANAGERPID: 3176
ORIGINAL_XDG_CURRENT_DESKTOP: pop

Processes (or workflows) are predefined applications which can be parametrised and executed by users. To get a list of currently available processes in our user workspace call Ades.get_processes() method:

In [3]:

for p in ades.get_processes():
    print(p.id)

echo
convert-url


You can fetch a specific workflow if you know it's ID using Ades.get_process() method. The Process object also contains metadata giving us more information about the process and how to execute it, for example the schema of inputs we can use to parametrise the process or output schema.

In [4]:
convert_url_proc = ades.get_process("convert-url")

pp(convert_url_proc.inputs_schema)
pp(convert_url_proc.outputs_schema)

{'fn': {'title': 'the operation to perform',
        'description': 'the operation to perform',
        'schema': {'type': 'string'}},
 'size': {'title': 'the percentage for a resize operation',
          'description': 'the percentage for a resize operation',
          'schema': {'type': 'string'}},
 'url': {'title': 'the image to convert',
         'description': 'the image to convert',
         'schema': {'type': 'string'}}}
{'converted_image': {'title': 'converted_image',
                     'description': 'None',
                     'extended-schema': {'oneOf': [{'allOf': [{'$ref': 'http://zoo-project.org/dl/link.json'},
                                                              {'type': 'object',
                                                               'properties': {'type': {'enum': ['application/json']}}}]},
                                                   {'type': 'object',
                                                    'required': ['value'],
                

In [ ]:
for j in ades.get_jobs():
    print(j.id, j.process_id, j.status)

Only one process with the same ID can exist. To demonstrate deploying a process further down in this notebook, we first need to undeploy convert-url. Note that attempting to delete a non-existent process will result in 4xx http status code.

In [ ]:
try:
    ades.get_process("convert-url").delete()
except HTTPError:
    print("Process not found, no need to undeploy.")

Let's deploy the convert-url process again. There are 2 ways we can provide the CWL file - either referencing the file by URL or by passing the CWL file content directly. Note that Ades.deploy_process() will fail if we try to create a process with ID that already exists. If we want to update an existing process, we should use Process.update() method instead. Both methods can handle URL or CWL YAML. In this example we deploy a process referencing by URL and then update it by passing the new CWL YAML content directly. Also note that when updating a worklow you need to provide the entire workflow, the API does not support partial updates (e.g. to change the description we need to provide the entire workflow again).

In [4]:
convert_url_proc = ades.deploy_process(
    cwl_url="https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-url-app.cwl"
)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL


In [5]:
cwl_yaml = """cwlVersion: v1.0
$namespaces:
  s: https://schema.org/
s:softwareVersion: 0.1.2
schemas:
  - http://schema.org/version/9.0/schemaorg-current-http.rdf
$graph:
  # Workflow entrypoint
  - class: Workflow
    id: convert-url
    label: convert url app
    doc: Convert URL YAML
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 1024
    inputs:
      fn:
        label: the operation to perform
        doc: the operation to perform
        type: string
      url:
        label: the image to convert
        doc: the image to convert
        type: string
      size:
        label: the percentage for a resize operation
        doc: the percentage for a resize operation
        type: string
    outputs:
      - id: converted_image
        type: Directory
        outputSource:
          - convert/results
    steps:
      convert:
        run: "#convert"
        in:
          fn: fn
          url: url
          size: size
        out:
          - results
  # convert.sh - takes input args `--url`
  - class: CommandLineTool
    id: convert
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 512
    hints:
      DockerRequirement:
        dockerPull: eoepca/convert:latest
    baseCommand: convert.sh
    inputs:
      fn:
        type: string
        inputBinding:
          position: 1
      url:
        type: string
        inputBinding:
          position: 2
          prefix: --url
      size:
        type: string
        inputBinding:
          position: 3
    outputs:
      results:
        type: Directory
        outputBinding:
          glob: .
"""

convert_url_proc.update(cwl_yaml=cwl_yaml)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL YAML


Let's execute our deployed process. We need to provide inputs as a dictionary in this format, see Process.inputs_schema property for inputs this particular workflow is expecting.

In [6]:
convert_url_job = convert_url_proc.execute(
    {
        "fn": "resize",
        "url": "https://eoepca.org/media_portal/images/logo6_med.original.png",
        "size": "50%",
    }
)

print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

672f0742-b6f5-11ef-a8b0-6a040e2afd6f running ZOO-Kernel accepted to run your service!


The job should now be running. Call Job.refresh() method to get the most up-to-date status and interrogate Job.status and Job.message properties. Note that these properties only hold the latest response from the API, and don't keep any historical records.

In [ ]:
convert_url_job.refresh()
print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

In [ ]:
We can continually poll the job using a simple loop and print status and message udpates like so:

from pyeodh.ades import AdesJobStatus
import time


old_status = ""
old_message = ""
while convert_url_job.status == AdesJobStatus.RUNNING.value:
    time.sleep(2)
    convert_url_job.refresh()
    if convert_url_job.status != old_status:
        print("\n")
        print(f"Status: {convert_url_job.status}")
    if convert_url_job.message != old_message:
        print(f"Message: {convert_url_job.message}")

    old_status = convert_url_job.status
    old_message = convert_url_job.message

In [ ]:
After the job has finished successfully, we can view the results, where the data files are referenced by assets

results = convert_url_job.get_result_items()
for res in results:
    print(res.id, res.assets)